In [1]:
import jax
import jax.numpy as jnp
from flax import linen as nn

import numpy as np
import matplotlib.pyplot as plt

import wavefunctions, hamiltonian, trajectory, optimization

import importlib, os

/home/amress/miniforge3/envs/nqs/lib/python3.9/site-packages/flax/struct.py:132: FutureWarning: jax.tree_util.register_keypaths is deprecated, and will be removed in a future release. Please use `register_pytree_with_keys()` instead.
  jax.tree_util.register_keypaths(data_clz, keypaths)
/home/amress/miniforge3/envs/nqs/lib/python3.9/site-packages/flax/struct.py:132: FutureWarning: jax.tree_util.register_keypaths is deprecated, and will be removed in a future release. Please use `register_pytree_with_keys()` instead.
  jax.tree_util.register_keypaths(data_clz, keypaths)


In [2]:
r_ws = 5.
N = 14
tau = 1.25

eqSteps = 1000
walkers = 8

L = wavefunctions.computeL(N, r_ws)

NUp = N // 2
NDown = N - NUp
spins = ( NUp , NDown )

In [4]:
importlib.reload(wavefunctions)

jax.config.update("jax_enable_x64", True)

###############################################################################
###  This tests the cusp conditions                                         ###
###############################################################################

blowupThreshold = 1e1
wfs = [
    wavefunctions.LogSlaterCYJastrow(spins, L),
    #wavefunctions.LogMessagePassingJastrow(
    #    spins, L, 3, 16, 16, 12, 4
    #),
    wavefunctions.LogTwoBodySJ(spins, L, 16)
]
dists = jnp.array([5e-3, 1e-3, 1e-4, 1e-5, 5e-6])

for wavefunction in wfs:
    
    localEnergy = hamiltonian.LocalEnergyUEG(wavefunction, L)
    
    rng = jax.random.PRNGKey(558)
    rng, inp_rng, init_rng = jax.random.split(rng, 3)
    np.random.seed(inp_rng)
    rs = jnp.array(L * np.random.rand(N,3))
    parameters = wavefunction.init(init_rng, rs)

    sameEnergies = np.full(dists.shape, np.nan)
    diffEnergies = np.full(dists.shape, np.nan)
    
    for (i,dist) in enumerate(dists):
        
        localWalker = np.array(rs)
        localWalker[1,:] = localWalker[0,:] + jnp.array([dist/2., dist/2., dist/2.])
        localWalker[0,:] = localWalker[0,:] - jnp.array([dist/2., dist/2., dist/2.])
        sameEnergies[i] = localEnergy.configuration(parameters, jnp.array(localWalker))
        
        localWalker = np.array(rs)
        localWalker[8,:] = localWalker[0,:] + jnp.array([dist/2., dist/2., dist/2.])
        localWalker[0,:] = localWalker[0,:] - jnp.array([dist/2., dist/2., dist/2.])
        diffEnergies[i] = localEnergy.configuration(parameters, jnp.array(localWalker))
    
    sameSuccess = np.max(sameEnergies) - np.min(sameEnergies) < blowupThreshold
    diffSuccess = np.max(diffEnergies) - np.min(diffEnergies) < blowupThreshold

    if sameSuccess and diffSuccess:
        print("Satisfies cusp conditions!")
    else:
        print("Failed cusp conditions...")
        print(sameEnergies)
        print(diffEnergies)

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


Satisfies cusp conditions!
Satisfies cusp conditions!


In [5]:
importlib.reload(wavefunctions)

wfs = [
    wavefunctions.LogSlaterCYJastrow(spins, L),
    #wavefunctions.LogMessagePassingJastrow(
    #    spins, L, 3, 16, 16, 12, 4
    #),
    wavefunctions.LogTwoBodySJ(spins, L, 16)
]

rng = jax.random.PRNGKey(558)
rng, inp_rng, init_rng = jax.random.split(rng, 3)
np.random.seed(inp_rng)
rs = jnp.array(L * np.random.rand(N,3))

for wavefunction in wfs:
        
    wavefunction = wavefunctions.LogSlaterCYJastrow(spins, L)
    parameters = wavefunction.init(init_rng, rs)
    result = wavefunctions.testAntisymmetry(wavefunction, parameters, rs)

    if result:
        print("Satisfies antisymmetry!")
    else:
        print("Failed antisymmetry...")

Satisfies antisymmetry!
Satisfies antisymmetry!
